In [3]:
import sqlite3

In [2]:
# Начало фрейма (frame_start) может быть:
#     current row — начиная с текущей строки;
#     N preceding — начиная с N-й строки перед текущей;
#     N following — начиная с N-й строки после текущей;
#     unbounded preceding — начиная с границы секции.

# Аналогично, конец фрейма (frame_end) может быть:
#     current row — до текущей строки;
#     N preceding — до N-й строки перед текущей;
#     N following — до N-й строки после текущей;
#     unbounded following — до границы секции.

In [4]:
connection = sqlite3.connect('window.db')
cursor = connection.cursor()

In [9]:
# Есть таблица сотрудников employees. Напишите запрос, который для каждого сотрудника выведет:

#     размер з/п предыдущего по зарплате сотрудника (среди коллег по департаменту);
#     максимальную з/п по департаменту.

# Если «предыдущего коллеги» нет (у сотрудника самая низкая зарплата в департаменте) — 
# запрос должен возвращать зарплату самого сотрудника.
# Не используйте функции lag() и lead().

cursor.execute('''
    SELECT id, name, department, salary, 
    FIRST_VALUE(salary) OVER w AS prev_salary,
    LAST_VALUE(salary) OVER w AS max_salary
    FROM employees
    WINDOW w AS (PARTITION BY department 
                ORDER BY salary, id
                ROWS BETWEEN 1 preceding AND unbounded following)
    ORDER BY department, salary, id
''')
print(*cursor.fetchall(), sep='\n')

(11, 'Дарья', 'hr', 70, 70, 78)
(12, 'Борис', 'hr', 78, 70, 78)
(21, 'Елена', 'it', 84, 84, 120)
(22, 'Ксения', 'it', 90, 84, 120)
(23, 'Леонид', 'it', 104, 90, 120)
(24, 'Марина', 'it', 104, 104, 120)
(25, 'Иван', 'it', 120, 104, 120)
(31, 'Вероника', 'sales', 96, 96, 100)
(32, 'Григорий', 'sales', 96, 96, 100)
(33, 'Анна', 'sales', 100, 96, 100)


In [ ]:
# ROWS BETWEEN frame_start AND frame_end
# GROUPS BETWEEN frame_start AND frame_end   - фреймы по группам
# RANGE BETWEEN frame_start AND frame_end    - фреймы по диапазону

In [ ]:
# Есть таблица сотрудников employees. 
# Для каждого человека мы хотим посчитать количество сотрудников, 
# которые получают такую же или большую зарплату (ge_cnt)

In [20]:
cursor.execute('''
    SELECT id, name, salary,
    COUNT(*) OVER w AS ge_cnt
    FROM employees
    WINDOW w AS (ORDER BY salary
                GROUPS BETWEEN current row AND unbounded following)
    ORDER BY salary, id
''')
print(*cursor.fetchall(), sep='\n')

(11, 'Дарья', 70, 10)
(12, 'Борис', 78, 9)
(21, 'Елена', 84, 8)
(22, 'Ксения', 90, 7)
(31, 'Вероника', 96, 6)
(32, 'Григорий', 96, 6)
(33, 'Анна', 100, 4)
(23, 'Леонид', 104, 3)
(24, 'Марина', 104, 3)
(25, 'Иван', 120, 1)


In [1]:
# Есть таблица сотрудников employees. Для каждого человека мы хотим увидеть ближайшую большую зарплату 

In [16]:
cursor.execute('''
    SELECT id, name, salary, 
        LAST_VALUE(salary) OVER w AS next_salary
    FROM employees
    WINDOW w AS (ORDER BY salary
                GROUPS BETWEEN 1 following AND 1 following)
    ORDER BY salary, id
''')
print(*cursor.fetchall(), sep='\n')

(11, 'Дарья', 70, 78)
(12, 'Борис', 78, 84)
(21, 'Елена', 84, 90)
(22, 'Ксения', 90, 96)
(31, 'Вероника', 96, 100)
(32, 'Григорий', 96, 100)
(33, 'Анна', 100, 104)
(23, 'Леонид', 104, 120)
(24, 'Марина', 104, 120)
(25, 'Иван', 120, None)
